# Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

## I. Camera Calibration

This section shows the steps to unwarp an image with lens and perspective distortions. These steps are:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images
* Apply a distortion correction to raw images
* Calculate the perspective transform according to the corners of undistorted images and some domain knowllege
* Use cv2.warpPerspective() to warp images to a top-down view

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

### 1.1 Compute the camera calibration matrix and distortion coefficients

#### 1.1.1 Prepare object points and image points

In [2]:
# Set the number of the chessboard corners
nx = 9
ny = 6

In [3]:
# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((ny*nx,3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

In [4]:
# Initialize arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

In [5]:
# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

In [6]:
# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    if img.dtype != 'uint8':
        img = np.uint8(img*255)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(200)
        
cv2.destroyAllWindows()

#### 1.1.2. Calculate the camera matrix and distortion coefficients

In [7]:
# Step through the list and search for chessboard corners use the function, cv2.calibrateCamera() 
# to calculate the camera matrix, distortion coefficients, rotation, and translation vectors etc.
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

### 1.2. Correct distortion

#### 1.2.1. Define a function to undistort an image

In [8]:
def img_undistort(img, mtx, dist):
    '''
    img: the raw image
    mtx: the camara matrix
    dist: the camara distortion coefficients
    '''
    if img.dtype != 'uint8':
        img = np.uint8(img*255)
    h, w = img.shape[:2]
    # Refine the camera matrix based on a free scaling parameter using cv2.getOptimalNewCameraMatrix()
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    # undistort
    undist = cv2.undistort(img, mtx, dist, None, newcameramtx)
    
    return undist, roi

#### 1.2.2. Test the calibration result using chessboard image

In [9]:
img = cv2.imread(images[0])
dst, roi = img_undistort(img, mtx, dist)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

### 1.3. Calculate the perspective transform

#### 1.3.1 Prepare src and dst points

In [10]:
img = cv2.imread(images[2])
# Undistort using mtx and dist
undist, roi = img_undistort(img, mtx, dist)
# Convert to grayscale
gray = cv2.cvtColor(undist, cv2.COLOR_BGR2GRAY)
# Find the chessboard corners
ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
# Given src and dst points
src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
# define 4 destination points dst = np.float32([[,],[,],[,],[,]])
offset = 100
dst = np.float32([[offset, offset], 
                  [offset+(nx-1)*offset-1, offset],
                  [offset+(nx-1)*offset-1, offset+(ny-1)*offset-1], 
                  [offset, offset+(ny-1)*100-1]])

#### 1.3.2 Get the transform matrix

In [11]:
# use cv2.getPerspectiveTransform() to get M, the transform matrix
M = cv2.getPerspectiveTransform(src, dst)

### 1.4. Warp images to a top-down view

#### 1.4.1. Define a function to unwarp an image

In [12]:
def img_unwarp(undist, M, size):
    '''
    undist: undistorted image
    M: the transform matrix
    size: the size of the unwarped image
    '''
    if undist.dtype != 'uint8':
        undist = np.uint8(img*255)
    unwarped = cv2.warpPerspective(undist, M, size, flags=cv2.INTER_LINEAR)
    
    return unwarped

#### 1.4.2. Test the function using chessboard image

In [13]:
# use cv2.warpPerspective() to warp the image to a top-down view
size = ((nx+1)*offset,(ny+1)*offset)
unwarped = img_unwarp(undist, M, size)

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(21, 6))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(undist)
ax2.set_title('Undistorted Image', fontsize=30)
ax3.imshow(unwarped)
ax3.set_title('Unwarped Image', fontsize=30)

## II. Pipeline (single images)

This section shows the steps for calculating the lane on a single image. These steps are:

* Get a distortion-corrected image.
* Used color transforms, gradients or other methods to create a thresholded binary image.
* Performed a perspective transform to the binary image.
* Identified lane-line pixels and fit their positions with a polynomial.
* Calculated the radius of curvature of the lane and the position of the vehicle with respect to center.
* Plot the result back down onto the road such that the lane area is identified clearly.

### 2.1 Create binary image

In [14]:
def SV_gray(img):
    '''convert BGR to HSV and merge the SV channels'''
    if img.dtype != 'uint8':
        img = np.uint8(img*255)
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    s_channel = np.float64(hsv_img[:,:,1])
    v_channel = np.float64(hsv_img[:,:,2])
    gray = np.uint8((0.2*s_channel+v_channel)/1.2)
    return gray

#### 2.1.1 Filter on directional gradient

In [15]:
# Define a function that takes an image, gradient orientation,
# and threshold min / max values.
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, abs_thresh=(0, 255)):
    if img.dtype != 'uint8':
        img = np.uint8(img*255)
    # Convert to grayscale
    gray = SV_gray(img)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.uint8(np.zeros_like(scaled_sobel))
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= abs_thresh[0]) & (scaled_sobel <= abs_thresh[1])] = 255

    # Return the result
    return binary_output

#### 2.1.2 Filter on the magnitude of the gradient

In [16]:
# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    if img.dtype != 'uint8':
        img = np.uint8(img*255)
    # Convert to grayscale
    gray = SV_gray(img)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.uint8(np.zeros_like(gradmag))
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 255

    # Return the binary image
    return binary_output

#### 2.1.3 Filter on the direction of the gradient

In [17]:
# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=3, dir_thresh=(0, np.pi/2)):
    if img.dtype != 'uint8':
        img = np.uint8(img*255)
    # Grayscale
    gray = SV_gray(img)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.uint8(np.zeros_like(absgraddir))
    binary_output[(absgraddir >= dir_thresh[0]) & (absgraddir <= dir_thresh[1])] = 255

    # Return the binary image
    return binary_output

#### 2.1.4 Filter on the S channel of the converted HSV image

In [18]:
# Define a function that thresholds the S-channel of HLS
# Use exclusive lower bound (>) and inclusive upper (<=)
def hls_threshold(img, hls_thresh=(0, 255)):
    if img.dtype != 'uint8':
        img = np.uint8(img*255)
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # 2) Apply a threshold to the S channel
    s_channel = hls[:,:,2]
    binary_output = np.uint8(np.zeros_like(s_channel))
    binary_output[(s_channel >= hls_thresh[0]) & (s_channel <= hls_thresh[1])] = 255
    # 3) Return a binary image of threshold result
    return binary_output

#### 2.1.5 Combining Thresholds

In [19]:
def combine_threshold(img, 
                      ksize=3, 
                      x_thresh=(0, 255), 
                      y_thresh=(0, 255), 
                      m_thresh=(0, 255), 
                      d_thresh=(0, np.pi/2), 
                      s_thresh=(0, 255)):
    if img.dtype != 'uint8':
        img = np.uint8(img*255)
    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(img, orient='x', sobel_kernel=ksize, abs_thresh=x_thresh)
    grady = abs_sobel_thresh(img, orient='y', sobel_kernel=ksize, abs_thresh=y_thresh)
    mag_binary = mag_thresh(img, sobel_kernel=ksize, mag_thresh=m_thresh)
    dir_binary = dir_threshold(img, sobel_kernel=ksize, dir_thresh=d_thresh)
    hls_binary = hls_threshold(img, hls_thresh=s_thresh)
    # Combine each channel
    combined = np.zeros_like(dir_binary)
    combined[(gradx == 255) & 
             (grady == 255) & 
             (mag_binary == 255) & 
             (dir_binary == 255) & 
             (hls_binary == 255)] = 255
    
    # Return the binary image
    return combined

#### 2.1.6 Test the function using an image

In [20]:
import matplotlib.image as mpimg

In [22]:
# Read in an image and grayscale it
image = mpimg.imread('./test_images/test2.jpg')

In [24]:
# Filter the image
bin_img = combine_threshold(image, 
                            ksize=3, 
                            x_thresh=(0, 255), 
                            y_thresh=(0, 255), 
                            m_thresh=(40, 255), 
                            d_thresh=(np.pi/20, np.pi/2-np.pi/20), 
                            s_thresh=(10, 255))

# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()

ax1.imshow(image)
ax1.set_title('Original Image', fontsize=40)

color_bin = np.dstack((bin_img, bin_img, bin_img))
ax2.imshow(color_bin)
ax2.set_title('Filtered Result', fontsize=40)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

### 2.2 Correct image distortion

#### 2.2.1 Grayscale an image with straight lines

In [26]:
# Read in an image and grayscale it
image = mpimg.imread('./test_images/straight_lines2.jpg')
bin_img = combine_threshold(image, 
                            ksize=3, 
                            x_thresh=(0, 255), 
                            y_thresh=(0, 255), 
                            m_thresh=(40, 255), 
                            d_thresh=(np.pi/20, np.pi/2-np.pi/20), 
                            s_thresh=(10, 255))
# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()

ax1.imshow(image)
ax1.set_title('Original Image', fontsize=40)

ax2.imshow(np.dstack((bin_img, bin_img, bin_img)))
ax2.set_title('Filtered Result', fontsize=40)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

#### 2.2.2 Undistort the binary image

In [27]:
undist, roi = img_undistort(bin_img, mtx, dist)

# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()

ax1.imshow(np.dstack((bin_img, bin_img, bin_img)))
ax1.set_title('Filtered Image', fontsize=40)
ax2.imshow(np.dstack((undist, undist, undist)))
ax2.set_title('Undistorted Image', fontsize=40)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

#### 2.2.3 Identify four source points for perspective transform

In [28]:
# define 4 source points src = np.float32([[,],[,],[,],[,]])
src = np.float32([[324,650], [1024,650], [712,450], [606,450]])
# define 4 destination points dst = np.float32([[,],[,],[,],[,]])
dst = np.float32([[color_bin.shape[1]/4,color_bin.shape[0]-1], 
                  [color_bin.shape[1]*3/4,color_bin.shape[0]-1], 
                  [color_bin.shape[1]*3/4,0], 
                  [color_bin.shape[1]/4,0]])

#### 2.2.4 Get the transform matrix

In [29]:
M = cv2.getPerspectiveTransform(src, dst)

#### 2.2.5 Unwarp the image

In [30]:
# use cv2.warpPerspective() to warp the image to a top-down view
size = (undist.shape[1],undist.shape[0])
unwarped = img_unwarp(undist, M, size)
unwarped = cv2.medianBlur(unwarped,5)
retval, unwarped = cv2.threshold(unwarped, 100, 255, cv2.THRESH_BINARY)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()

undist_plus = np.dstack((undist, undist, undist))
undist_plus = cv2.polylines(undist_plus,np.int32([src]),True,(255,0,0),2)
ax1.imshow(undist_plus)
ax1.set_title('Undistorted Image', fontsize=40)
unwarped_plus = np.dstack((unwarped, unwarped, unwarped))
unwarped_plus = cv2.polylines(unwarped_plus,np.int32([dst]),True,(255,0,0),2)
ax2.imshow(unwarped_plus)
ax2.set_title('Unwarped Image', fontsize=40)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

### 2.3 Identified lane-line pixels

#### 2.3.1 Find peaks in the Histogram

In [31]:
# take a histogram along all the columns in the lower half of the image
histogram = np.sum(unwarped[np.int32(unwarped.shape[0]/2):,:], axis=0)
plt.plot(histogram)

In [32]:
# Find the peak of the left and right halves of the histogram
# These will be the starting point for the left and right lines
midpoint = np.int(histogram.shape[0]/2)
leftx_base = np.argmax(histogram[:midpoint])
rightx_base = np.argmax(histogram[midpoint:]) + midpoint

#### 2.3.2 Implement Sliding Windows

In [33]:
# Create an output image to draw on and  visualize the result
out_img = np.dstack((unwarped, unwarped, unwarped))
# Choose the number of sliding windows
nwindows = 9
# Set height of windows
window_height = np.int(unwarped.shape[0]/nwindows)
# Identify the x and y positions of all nonzero pixels in the image
nonzero = unwarped.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
# Current positions to be updated for each window
leftx_current = leftx_base
rightx_current = rightx_base
# Set the width of the windows +/- margin
margin = 100
# Set minimum number of pixels found to recenter window
minpix = 20
# Create empty lists to receive left and right lane pixel indices
left_lane_inds = []
right_lane_inds = []

In [34]:
# Step through the windows one by one
for window in range(nwindows):
    # Identify window boundaries in x and y (and right and left)
    win_y_low = unwarped.shape[0] - (window+1)*window_height
    win_y_high = unwarped.shape[0] - window*window_height
    win_xleft_low = leftx_current - margin
    win_xleft_high = leftx_current + margin
    win_xright_low = rightx_current - margin
    win_xright_high = rightx_current + margin
    # Draw the windows on the visualization image
    cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
    cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
    # Identify the nonzero pixels in x and y within the window
    good_left_inds = ((nonzeroy >= win_y_low) & 
                      (nonzeroy < win_y_high) & 
                      (nonzerox >= win_xleft_low) & 
                      (nonzerox < win_xleft_high)).nonzero()[0]
    good_right_inds = ((nonzeroy >= win_y_low) & 
                       (nonzeroy < win_y_high) & 
                       (nonzerox >= win_xright_low) & 
                       (nonzerox < win_xright_high)).nonzero()[0]
    # Append these indices to the lists
    left_lane_inds.append(good_left_inds)
    right_lane_inds.append(good_right_inds)
    # If you found > minpix pixels, recenter next window on their mean position
    if len(good_left_inds) > minpix:
        leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
    if len(good_right_inds) > minpix:        
        rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

### 2.4 Fit lane-line pixels positions with a polynomial

In [35]:
# Concatenate the arrays of indices
left_lane_inds = np.concatenate(left_lane_inds)
right_lane_inds = np.concatenate(right_lane_inds)

# Extract left and right line pixel positions
leftx = nonzerox[left_lane_inds]
lefty = nonzeroy[left_lane_inds] 
rightx = nonzerox[right_lane_inds]
righty = nonzeroy[right_lane_inds] 

# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)

In [36]:
# Generate x and y values for plotting
ploty = np.linspace(0, unwarped.shape[0]-1, unwarped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
plt.imshow(out_img)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)

(720, 0)

### 2.5 Skip the sliding windows step

In [37]:
# Assume you now have a new warped binary image 
# from the next frame of video (also called "binary_warped")
# It's now much easier to find line pixels!
nonzero = unwarped.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & 
                  (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & 
                   (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

# Again, extract left and right line pixel positions
leftx = nonzerox[left_lane_inds]
lefty = nonzeroy[left_lane_inds] 
rightx = nonzerox[right_lane_inds]
righty = nonzeroy[right_lane_inds]
# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)
# Generate x and y values for plotting
ploty = np.linspace(0, unwarped.shape[0]-1, unwarped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

In [38]:
# Create an image to draw on and an image to show the selection window
out_img = np.dstack((unwarped, unwarped, unwarped))
window_img = np.zeros_like(out_img)
# Color in left and right line pixels
out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

# Generate a polygon to illustrate the search window area
# And recast the x and y points into usable format for cv2.fillPoly()
left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
left_line_pts = np.hstack((left_line_window1, left_line_window2))
right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
right_line_pts = np.hstack((right_line_window1, right_line_window2))

# Draw the lane onto the warped blank image
cv2.fillPoly(window_img, np.int_([left_line_pts]), (0, 255, 0))
cv2.fillPoly(window_img, np.int_([right_line_pts]), (0, 255, 0))
result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
plt.imshow(result)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)

(720, 0)

### 2.6 Calculated the radius of curvature

In [39]:
# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.6/640 # meters per pixel in x dimension

# Fit new polynomials to x,y in world space
left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
# Calculate the new radii of curvature
y_eval = np.max(ploty)
left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
# Now our radius of curvature is in meters
print(left_curverad, 'm', right_curverad, 'm')

4341.7829328 m 24824.1472069 m


### 2.7 Calculated the position of the vehicle

In [40]:
# Calculate the position of the left line and right line at the bottom of the image
left_center_x = left_fit[0]*unwarped.shape[0]**2 + left_fit[1]*unwarped.shape[0] + left_fit[2]
right_center_x = right_fit[0]*unwarped.shape[0]**2 + right_fit[1]*unwarped.shape[0] + right_fit[2]
# conversions from pixels space to meters and calculate the offset from the center
offset_cr = (unwarped.shape[1]/2-(left_center_x+right_center_x)/2)*xm_per_pix
print(left_center_x,right_center_x)
print('The car is', offset_cr, 'm from the center of line.')

320.592036163 959.657366926
The car is -0.000701446188426 m from the center of line.


### 2.8 Plot the result back down onto the road

In [41]:
# Create an image to draw the lines on
warp_zero = np.zeros_like(unwarped).astype(np.uint8)
color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

# Recast the x and y points into usable format for cv2.fillPoly()
pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
pts = np.hstack((pts_left, pts_right))

# Draw the lane onto the warped blank image
cv2.fillPoly(color_warp, np.int_([pts]), (0,255,0))

# Warp the blank back to original image space using inverse perspective matrix (Minv)
Minv = np.linalg.inv(M)
newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
# Combine the result with the original image (undistorted)
undist_img, roi = img_undistort(image, mtx, dist)
result = cv2.addWeighted(undist_img, 0.7, newwarp, 0.3, 0)
x,y,w,h = roi
result = result[y:y+h, x:x+w]
cv2.putText(result, 
            'Radius of Curvature = {} (m)'.format(int((left_curverad+right_curverad)/2)), 
            org = (30,50), 
            fontFace = cv2.FONT_HERSHEY_SIMPLEX, 
            fontScale = 2, 
            color = (255,255,255), 
            thickness = 2)
cv2.putText(result, 
            'Vehicle is {:03.2f}m left of center'.format(offset_cr), 
            org = (30,100), 
            fontFace = cv2.FONT_HERSHEY_SIMPLEX, 
            fontScale = 2, 
            color = (255,255,255), 
            thickness = 2)
plt.imshow(result)

## III. Pipeline (video)

### 3.1 Wrap up all steps above

In [42]:
def pipline(image, mtx, dist, M, 
            ksize=3, 
            x_thresh=(0, 255), 
            y_thresh=(0, 255), 
            m_thresh=(40, 255), 
            d_thresh=(np.pi/20, np.pi/2-np.pi/20), 
            s_thresh=(10, 255), 
            nwindows = 9, 
            margin = 100, 
            minpix = 10, 
            left_fit = None, 
            right_fit = None, 
            ym_per_pix = 30/720, 
            xm_per_pix = 3.6/640, 
            decay_factor = 0.9):
    bin_img = combine_threshold(image, 
                            ksize=ksize, 
                            x_thresh=x_thresh, 
                            y_thresh=y_thresh, 
                            m_thresh=m_thresh, 
                            d_thresh=d_thresh, 
                            s_thresh=s_thresh)
    undist, roi = img_undistort(bin_img, mtx, dist)
    unwarped = img_unwarp(undist, M, (undist.shape[1],undist.shape[0]))
    unwarped = cv2.medianBlur(unwarped,5)
    retval, unwarped = cv2.threshold(unwarped, 100, 255, cv2.THRESH_BINARY)
    # Line detection
    if left_fit == None or right_fit == None:
        histogram = np.sum(unwarped[np.int32(unwarped.shape[0]/2):,:], axis=0)
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
        # Set height of windows
        window_height = np.int(unwarped.shape[0]/nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = unwarped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []
        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = unwarped.shape[0] - (window+1)*window_height
            win_y_high = unwarped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & 
                              (nonzeroy < win_y_high) & 
                              (nonzerox >= win_xleft_low) & 
                              (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & 
                               (nonzeroy < win_y_high) & 
                               (nonzerox >= win_xright_low) & 
                               (nonzerox < win_xright_high)).nonzero()[0]
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    else:
        # Assume you now have a new warped binary image 
        # from the next frame of video (also called "binary_warped")
        # It's now much easier to find line pixels!
        nonzero = unwarped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & 
                          (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & 
                           (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    if len(leftx) > minpix*5:
        if left_fit != None:
            left_fit = np.polyfit(lefty, leftx, 2)*(1-decay_factor) + left_fit*decay_factor
        else:
            left_fit = np.polyfit(lefty, leftx, 2)
    if len(rightx) > minpix*5:
        if right_fit != None:
            right_fit = np.polyfit(righty, rightx, 2)*(1-decay_factor) + right_fit*decay_factor
        else:
            right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, unwarped.shape[0]-1, unwarped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Calculate the position of the left line and right line at the bottom of the image
    left_center_x = left_fit[0]*unwarped.shape[0]**2 + left_fit[1]*unwarped.shape[0] + left_fit[2]
    right_center_x = right_fit[0]*unwarped.shape[0]**2 + right_fit[1]*unwarped.shape[0] + right_fit[2]
    # conversions from pixels space to meters and calculate the offset from the center
    offset_cr = (unwarped.shape[1]/2-(left_center_x+right_center_x)/2)*xm_per_pix
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(unwarped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255,0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    Minv = np.linalg.inv(M)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image (undistorted)
    undist_img, roi = img_undistort(image, mtx, dist)
    result = cv2.addWeighted(undist_img, 0.7, newwarp, 0.3, 0)
    x,y,w,h = roi
    result = result[y:y+h, x:x+w]
    cv2.putText(result, 
                'Radius of Curvature = {} (m)'.format(int((left_curverad+right_curverad)/2)), 
                org = (30,50), 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, 
                fontScale = 2, 
                color = (255,255,255), 
                thickness = 2)
    cv2.putText(result, 
                'Vehicle is {:03.2f}m right of center'.format(offset_cr), 
                org = (30,100), 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, 
                fontScale = 2, 
                color = (255,255,255), 
                thickness = 2)
    return result, left_fit, right_fit

In [43]:
cap = cv2.VideoCapture('./project_video.mp4')

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v') # note the lower case
out = cv2.VideoWriter('./project_output.avi',fourcc, 30, (1200,617))

counter = 1
left_fit = None
right_fit = None
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        #cv2.imwrite('../clips/{}.jpg'.format(counter),frame)
        result, left_fit, right_fit = pipline(frame, mtx, dist, M, 
                                                ksize=3, 
                                                x_thresh=(0, 255), 
                                                y_thresh=(0, 255), 
                                                m_thresh=(30, 255), 
                                                d_thresh=(np.pi/20, np.pi/2-np.pi/20), 
                                                s_thresh=(30, 255), 
                                                nwindows = 9, 
                                                margin = 80, 
                                                minpix = 20, 
                                                left_fit = left_fit,
                                                right_fit = right_fit, 
                                                ym_per_pix = 50/720, 
                                                xm_per_pix = 3.6/640, 
                                                decay_factor = 0.9)
        #cv2.imwrite('../clips/result{}.jpg'.format(counter),result)
        out.write(result)
        counter += 1
        # Display the resulting frame
        cv2.imshow('result',result)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# When everything done, release the capture
cap.release()
out.release()
cv2.destroyAllWindows()

/Users/libaoling/anaconda/envs/OpenCV-py3-cv3/lib/python3.5/site-packages/ipykernel/__main__.py:28: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/Users/libaoling/anaconda/envs/OpenCV-py3-cv3/lib/python3.5/site-packages/ipykernel/__main__.py:93: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/Users/libaoling/anaconda/envs/OpenCV-py3-cv3/lib/python3.5/site-packages/ipykernel/__main__.py:98: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
